# Assignment 1

# 4. Exact Diagonalization study of the quantum Ising model

# 4.1 Dense ED

Generate the quantum Ising Hamiltonian (4) as a dense matrix and call an explicit diagonalization routine for the entire spectrum for system sizes $L = 8, 10, 12, 14,$ and for a range of values of h. Plot the ground state energy as a function of $h$ for the various $L$. Compare the open systems with periodic ones for the same parameters—how does each phase react to the boundaries?


import relevant packages

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
from tqdm import tqdm
import time
import sys
import os
from multiprocessing import Pool
from functools import partial
directory = 'figures'
if not os.path.exists(directory):
    os.makedirs(directory)
from concurrent.futures import ProcessPoolExecutor, as_completed
from tqdm.notebook import tqdm
import numpy as np
from dask.distributed import Client, progress
from dask import compute, delayed
import dask.array as da
from dask.diagnostics import ProgressBar

## Constructing the  dense Hamiltonian

The Hamiltonian of the quantum Ising chain is given by

$$H = -J \sum_{j=1}^{L-1} \sigma_z^j \sigma_z^{j+1} - h \sum_{j=1}^{L} \sigma_x^j $$

Instead of having to do all these time-consuming Kronecker products we can quickly realize that 

$$		H_{\alpha \beta} = \langle e_\alpha | H | e_\beta \rangle \neq 0 \quad \text{if} \quad 
		\begin{cases}
			\alpha = \beta \\
			\text{or } \alpha \text{ and } \beta \text{ differ by a single bit flip.}
		\end{cases}$$
        
This is because $|\uparrow \rangle$ and $|\downarrow \rangle$ are eigenstates of $\sigma_z$ and $\sigma_x$ flips $|\uparrow \rangle \longleftrightarrow |\downarrow \rangle$. The states $| e_\alpha \rangle$  are defined in an orthogonal basis as follows:


\begin{align*}
|e_0\rangle &= |000 \ldots 0\rangle, \\
|e_1\rangle &= |100 \ldots 0\rangle, \\
|e_2\rangle &= |010 \ldots 0\rangle, \\
|e_3\rangle &= |110 \ldots 0\rangle, \\
&\text{and so on.}
\end{align*}


Where $0$ represents $|\uparrow \rangle$ and $1$ represents $|\downarrow \rangle$. Now that we have the states represented as integers, we can use fast bit operations to quickly construct the Hamiltonian.

In [ ]:
def denseH(L, J, h, periodic):
    """
    generates the dense Hamiltonian matrix for the quantum Ising chain
    
        Parameters:
            L (int): length of chain
            J (float): ising interaction strength
            h (float): magnetic field strength
            periodic (bool): does the chain have periodic boundary conditions?
            
        Returns:
            H (ndarray): 2^L x 2^L matrix representing the Hamiltonian operator
    """

    dim=2 ** L # dimensions of the Hilbert space
    
    H=np.zeros((dim, dim)) # initliaze the Hamiltonian
    
    "Calculation of off-diagonal elements due to the magnetic field"
    
    for beta in range(dim): # iterate over all states
        
        for j in range(1,L+1): # iterate over all sites
            
            alpha = beta ^ (1<<j-1) # flips jth bit of beta to get the state alpha that is related to beta by a single bit flip
            
            H[alpha, beta] -= h # contribution by sigma^j_x
            
    "Calculation of diagonal elements due to Ising interaction"

    for alpha in range(dim): # iterate over all states
        
        for j in range(1, L): # iterate over all sites
            
            if 2*(alpha & (1 << j-1)) == alpha & (1 << j): # check if site j and j+1 have the same spin
                
                H[alpha, alpha] -= J # if they do, decrease the energy by the ising interaction term
                
            else:
                
                H[alpha, alpha] += J # if not, increase the energy by the ising interaction term
        
        "Handling case of periodic boundary conditions"
                
        if periodic and L > 1: # L > 1 needed for periodicity to mean anything
            
            if (alpha & (1 << L-1)) == ((alpha & (1 << 0))*(2**(L-1))): # Check if the states at either end have the same spin
                
                H[alpha, alpha] -= J # if they do, decrease the energy by the ising interaction term
                
            else:
                
                H[alpha, alpha] += J # if not, increase the energy by the ising interaction term
                
    return H   

### Dense diagonalization implementation

In [ ]:
def denseEgs(L, J, h, periodic):
    """
    returns the ground state eigenenergy of the Ising chain
    
        Parameters:
            L (int): length of chain
            J (float): ising interaction strength
            h (float): magnetic field strength
            periodic (bool): does the chain have periodic boundary conditions?
        
        Returns:
            ground_state (float): ground state energy
    """

    H = denseH(L, J, h , periodic) # construct the dense Hamiltonian
    
    ground_state = scipy.linalg.eigh(H, subset_by_index=(0, 0), eigvals_only=True)[0] # return only the smallest eigenvalue (increases the speed quite a bit)
    
    return ground_state 

### Plotting the ground state energy as a function of h for various L

In [ ]:
# initialize
L_values = [8] 
h_values = np.linspace(0, 2, 20) # the problem is symmetric about h = 0, so we can cut computation-time in half by computing only positive h
periodicgs = {L: [] for L in L_values}
opengs = {L: [] for L in L_values}
xkcd_colors=['xkcd:indigo', 'xkcd:royal blue', 'xkcd:bright green', 'xkcd:red']

"Plot Egs vs h for both periodic and open boundary conditions"

for L in L_values:
    for h in tqdm(h_values, desc=f'Calculating for L={L}'):
        periodicgs[L].append(denseEgs(L, 1, h, True))
        opengs[L].append(denseEgs(L, 1, h, False))
        
# Dashed line -> Open
# Solid line -> Periodic
        
for i, (L, energies) in enumerate(periodicgs.items()):
    plt.plot(h_values, energies, label=f'L={L}', color=xkcd_colors[i])
    
for i, (L, energies) in enumerate(opengs.items()):
    plt.plot(h_values, energies, color=xkcd_colors[i], linestyle='--')

plt.xlabel('field strength')
plt.ylabel(r'$E_{gs}$')
plt.title('Ground state energy versus magnetic field \n for various values of L')
plt.legend()
plt.savefig(os.path.join(directory, 'plot-dense.png'), dpi=400)
plt.show()

# 4.2 Sparse ED

Construct the same Hamiltonian as a sparse matrix and use a sparse matrix diagonalization routine
provided by a standard library. Obtain the ground state and a few excited states, and for small
system sizes verify your results against the dense ED solution. How large of a system can you push
the sparse diagonalization routine to solve?

Optional. Try implementing your own Lanczos routine.

As you can see L = 14 takes forever to run and the memory limits significantly constrain us as well. But, most of the elements of the Hamiltonian are zero, we can use this sparseness to our advantage by slightly modifying our code. Instead of initializing the whole $2^L \times 2^L$ matrix, we just keep track of the non-zero elements of the Hamiltonian and use scipy.sparse to convert it into a standard sparse-matrix data structure.

## Constructing the Sparse Hamiltonian

In [ ]:
def sparseH(L, J, h, periodic):
    
    """
    generates the sparse Hamiltonian matrix for the quantum Ising chain
    
        Parameters:
            L (int): length of chain
            J (float): ising interaction strength
            h (float): magnetic field strength
            periodic (bool): does the chain have periodic boundary conditions?
            
        Returns:
            H (csr_matrix): sparse matrix representing the Hamiltonian operator
    """
    
    dim = 2 ** L # dimensions of the Hilbert space
    
    # initialize 
    H_data = []
    H_rows = []
    H_cols = []
    
    "Calculation of off-diagonal elements due to the magnetic field"
    
    for beta in range(dim): # iterate over all states
        
        for j in range(1, L + 1): # iterate over all sites
            
            alpha = beta ^ (1 << (j - 1)) # flips jth bit of beta to get the state alpha that is related to beta by a single bit flip
            
            "Keep track of the indices with non-zero matrix elements"
            
            H_data.append(-h)
            H_rows.append(alpha)
            H_cols.append(beta)
    
    "Calculation of diagonal elements due to Ising interaction"

    for alpha in range(dim):  # iterate over all states
        
        A = 0
        
        for j in range(1, L): # iterate over all sites
            
            if 2 * (alpha & (1 << (j - 1))) == alpha & (1 << j): # check if site j and j+1 have the same spin
                
                A -= J # if they do, decrease the energy by the ising interaction term
                
            else:
                
                A += J # if not, increase the energy by the ising interaction term
                
        "Handling periodic boundary conditions"
                
        if periodic and L > 1: # L > 1 needed for periodicity to mean anything
            
            if (alpha & (1 << (L - 1))) == ((alpha & (1 << 0)) * (2 ** (L - 1))): # Check if the states at either end have the same spin
                
                A -= J # if they do, decrease the energy by the ising interaction term
                
            else:
                
                A += J # if not, increase the energy by the ising interaction term
        
        if A != 0: # Check if the resulting matrix element is non-zero, if so, keep track of it
        
            H_data.append(A)
            H_rows.append(alpha)
            H_cols.append(alpha)

    H_data = np.array(H_data, dtype=float) # convert the list into a np array
    
    H = scipy.sparse.csr_matrix((H_data, (H_rows, H_cols)), shape=(dim, dim), dtype=np.float64) # make it into a csr sparse matrix
    
    return H

## Consistency between Sparse and Dense Diagonalizers

Let me write up some useful functions

In [ ]:
def diagonalize_dense(L, J, h, periodic):
    """
    generates the eigenstates and eigenenergies from the dense ED
    
        Parameters:
            L (int): length of chain
            J (float): ising interaction strength
            h (float): magnetic field strength
            periodic (bool): does the chain have periodic boundary conditions?
            
        Returns:
            energies: eigenenergies
            states: eigenvectors
    """

    energies, states = scipy.linalg.eigh(denseH(L, J, h, periodic))
    return energies, states

def diagonalize_sparse(L, J, h, periodic, num_states=6):
    """
    generates the eigenstates and eigenenergies from the dense ED
    
        Parameters:
            L (int): length of chain
            J (float): ising interaction strength
            h (float): magnetic field strength
            periodic (bool): does the chain have periodic boundary conditions?
            num_states = number of states needed
            
        Returns:
            energies: eigenenergies
            states: eigenvectors
    """
    energies, states = scipy.sparse.linalg.eigsh(sparseH(L, J, h, periodic), k=num_states, which='SA')
    return energies, states

def compare_states(states_dense, states_sparse):
    """
    computes the overlap between corresponding eigenstates obtained from dense and sparse diagonalization methods.
    
    Parameters:
        states_dense (np.ndarray): a 2D array containing the eigenvectors obtained from the dense diagonalization.

        states_sparse (np.ndarray): a 2D array containing the eigenvectors obtained from the sparse diagonalization.
                                    
    Returns:
        overlaps (list): list of overlap values for corresponding eigenstates. Each overlap value is a float between
                         0 and 1.
    """
    
    overlaps = []
    
    for i in range(len(states_sparse[0])):
        overlap = abs(np.dot(states_dense[:, i].conjugate(), states_sparse[:, i])) 
        # calculates the absolute overlap between the i-th eigenvectors from the dense and sparse results
        
        overlaps.append(overlap)
        
    return overlaps

In [ ]:
L_values = [8, 10]
h_values = np.linspace(0, 1, 5)
num_states = 6

for L in L_values:
    for h in h_values:
        
        energies_dense, states_dense = diagonalize_dense(L, 1, h, True)
        energies_sparse, states_sparse = diagonalize_sparse(L, 1, h, True, num_states=num_states)
        
        print(f"L={L}, h={h}")
        print("Dense ED Energies:", energies_dense[:num_states])
        print("Sparse ED Energies:", energies_sparse)
        
        overlaps = compare_states(states_dense, states_sparse)
        print("State overlaps:", overlaps)


As you can see the eigenenergies do match, but the eigenvectors do not seem to have perfect overlap, this is because the states are degenerate and any linear combination can be reported as an eigenvector. You can see this clearly as the non-degenerate states do seem to match.

## Implementation of the sparse matrix as a functional pointer 

The code below might be useful if there are memory constraints, as instead of writing the Hamiltonian as a matrix we just write up rules that give $H | \psi \rangle$ given $| \psi \rangle$. This method however seems to be less time-efficient to diagonalize.

In [ ]:
def H_operator(L, J, h, periodic):
    """
    generates the Hamiltonian operator for the quantum Ising chain
    
        Parameters:
            L (int): length of chain
            J (float): ising interaction strength
            h (float): magnetic field strength
            periodic (bool): does the chain have periodic boundary conditions?
            
        Returns:
            H (LinOperator): function pointer representing the Hamiltonian operator
    """
    def H_psi(psi):
        """
        Generates Hpsi given psi
        """
        dim = 2 ** L
        Hpsi = np.zeros(dim, dtype=np.float64)
        
        
        "Calculation of off-diagonal elements due to the magnetic field"
        
        for beta in range(dim): # iterate over all states
        
            for j in range(1, L + 1): # iterate over all sites
                
                alpha = beta ^ (1 << (j - 1)) # flips jth bit of beta to get the state alpha that is related to beta by a single bit flip
                
                Hpsi[alpha] -= h * psi[beta] # change the coefficient of the alpha state
                
        
        "Calculation of diagonal elements due to Ising interaction"
        
        for alpha in range(dim): # iterate over all states
            
            A = 0

            for j in range(1, L): # iterate over all sites

                if 2 * (alpha & (1 << (j - 1))) == alpha & (1 << j):  # check if site j and j+1 have the same spin
                    
                    A -= J # if they do, decrease the energy by the ising interaction term
                    
                else:
                    A += J # if not, increase the energy by the ising interaction term

            if periodic and L > 1:  # L > 1 needed for periodicity to mean anything
                
                if (alpha & (1 << (L - 1))) == ((alpha & (1 << 0)) * (2 ** (L - 1))): # Check if the states at either end have the same spin
                    
                    A -= J # if they do, decrease the energy by the ising interaction term
                    
                else:
                    A += J # if not, increase the energy by the ising interaction term

            Hpsi[alpha] += A * psi[alpha] # change the coefficient of the alpha state
    
    dim = 2 ** L
    
    return scipy.sparse.linalg.LinearOperator((dim, dim), matvec=H_psi)


Performance difference between the two styles

In [ ]:
L = 14
J = 1
h = 0.5
periodic = True
num_states = 1


start_time_sparse = time.time()
H_sparse = sparseH(L, J, h, periodic)
energies_sparse, states_sparse = scipy.sparse.linalg.eigsh(H_sparse, k=num_states, which='SA')
end_time_sparse = time.time()

start_time_operator = time.time()
H_op = H_operator(L, J, h, periodic)
energies_op, states_op = scipy.sparse.linalg.eigsh(H_op, k=num_states, which='SA')
end_time_operator = time.time()

time_sparse = end_time_sparse - start_time_sparse
time_operator = end_time_operator - start_time_operator

print('time taken by csr Hamiltonian: ', time_sparse)

print('time taken by operator Hamiltonian: ', time_operator)



H_sparse_memory = sys.getsizeof(H_sparse.data) + sys.getsizeof(H_sparse.indices) + sys.getsizeof(H_sparse.indptr)
H_op_memory = sys.getsizeof(H_op)

print('memory used by csr Hamiltonian: ', H_sparse_memory)

print('memory used by operator Hamilonian: ', H_op_memory)


Since time is our bigger concern I will use sparseH, however this functional pointer approach might be worth considering depending on the task chosen.

# 4.3 Study of convergence with system size

For representative values of $h$ inside each phase (e.g., $h = 0.3$) in the ferromagnetic phase and $h = 1.7$ in the paramagnet), study the $L$ dependence of the ground state energy per site, $E_{\text{gs}}(L)/L$, for systems with both periodic and open boundary conditions. Comment on the approach to the thermodynamic limit $L \rightarrow \infty$ for the two types of boundaries.



In [ ]:
def sparseEgs(L, J, h, periodic):
    """
    returns the ground state eigenenergy of the Ising chain using sparse diagonalization 
    
        Parameters:
            L (int): length of chain
            J (float): ising interaction strength
            h (float): magnetic field strength
            periodic (bool): does the chain have periodic boundary conditions?
        
        Returns:
            ground_state (float): ground state energy
    """

    H = denseH(L, J, h , periodic) # construct the dense Hamiltonian
    
    ground_state = scipy.sparse.linalg.eigsh(H, k=1, which='SA', return_eigenvectors=False)[0] # return only the smallest eigenvalue (increases the speed quite a bit)
    
    return ground_state 

Let us check the convergence to thermodynamic limit of the ferromagnetic and paramagnetic phases for increasing L

In [ ]:
paraPERIODIC=[]
ferroPERIODIC=[]
paraOPEN=[]
ferroOPEN=[]
Ls = list(range(2, 25))  

for L in tqdm(Ls, desc="Calculating energies"):
    start_time = time.time()
    ferroPERIODIC.append(sparseEgs(L, 1, 0.3, True)/L)
    paraPERIODIC.append(sparseEgs(L, 1, 1.7, True)/L)
    ferroOPEN.append(sparseEgs(L, 1, 0.3, False)/L)
    paraOPEN.append(sparseEgs(L, 1, 1.7, False)/L)
    elapsed_time = time.time() - start_time
    print(f"L = {L}, Time taken: {elapsed_time:.2f} seconds")

In [ ]:
plt.plot(Ls, ferroOPEN, label='open', linestyle="dashdot", color='xkcd:royal blue')
plt.plot(Ls, ferroPERIODIC, label="periodic", color="xkcd:bright red")
plt.legend()
plt.xlabel('L')
plt.ylabel(r'$\epsilon_{gs}(L)/L$')
plt.title( r'$\epsilon_{gs}(L)/L$ dependence on $L$ for ferro-magnetic phase ($h=0.3$)')
plt.savefig(os.path.join(directory, 'ferro_open_periodic.png'), dpi=400)
plt.show()



plt.plot(Ls, paraOPEN, label='open', linestyle="dashdot", color='xkcd:royal blue')
plt.plot(Ls, paraPERIODIC, label="periodic", color="xkcd:bright red")
plt.title( r'$\epsilon_{gs}(L)/L$ dependence on $L$ for para-magnetic phase ($h=1.7$)')
plt.xlabel('L')
plt.ylabel(r'$\epsilon_{gs}(L)/L$')
plt.legend()
plt.savefig(os.path.join(directory, 'para_open_periodic.png'), dpi=400)
plt.show()

# 4.3 Finding the quantum phase transition

## Excitation gap curve

I will be using the dask package to speed up the computations so that it doesn't take unreasonably long time.

In [ ]:
client = Client(n_workers=20)
ProgressBar().register()

In [ ]:
@delayed

# function that can be inputted into dash
def diagonalize_h(h):
    H = sparseH(L, J, h, periodic)
    eigs = scipy.sparse.linalg.eigsh(H, k=num_states, which='SA', return_eigenvectors=False)
    return h, np.sort(eigs)

L = 20 # could have selected higher value, but it takes really long
J = 1.0
h_values = np.linspace(0, 2, 50)
periodic = True
num_states = 10 # more states can be selected
energies = np.zeros((len(h_values), num_states))

# Create a list of delayed tasks
tasks = [diagonalize_h(h) for h in h_values]

# Compute tasks with progress bar
results = compute(*tasks, scheduler='processes')

# Populate the energies array
for h, eigs in results:  # Iterate directly over results
    index = np.where(h_values == h)[0][0]
    energies[index, :] = eigs



### Plot of Excited state energies vs field strength

Plotting the first 9 excited staes/

In [ ]:
excitation_energies = energies[:, 1:] - energies[:, 0, np.newaxis]
    
plt.figure(figsize=(10, 8))
plt.rcdefaults()
plt.rcParams['figure.dpi'] = 400

# Define line styles and markers
line_styles = ['-', '--', '-.', ':']
markers = ['o', '^', 's', '*', 'p', 'D', 'x', '+']

for i in range(1, num_states):
    plt.plot(h_values, excitation_energies[:, i-1], 
             label=f'Excited State {i}',
             linestyle=line_styles[i % len(line_styles)],  # Cycle through line styles
             marker=markers[i % len(markers)],  # Cycle through markers
             linewidth=1.5,
             markersize=4,
             alpha=0.8)
plt.xlabel('$h$')
plt.ylabel(r'$\varepsilon_{\text{excited}}$-$\varepsilon_{\text{gs}}$')
plt.title('Excitation Energies relative to ground-state vs. Magnetic Field Strength')
plt.savefig(os.path.join(directory, 'excited_states2.png'), dpi=400)
plt.show()


## Isolating the first two excited states

In [ ]:
h_values = np.linspace(0, 2, 50)

# Define line styles and markers
line_styles = ['-', '--', '-.', ':']
markers = ['o', '^', 's', '*', 'p', 'D', 'x', '+']


plt.plot(h_values-1, excitation_energies[:, 1-1], 
             label=f'Excited State {1}',
             linestyle=line_styles[1 % len(line_styles)],  # Cycle through line styles
             marker=markers[1 % len(markers)],  # Cycle through markers
             linewidth=1.5,
             markersize=4,
             alpha=0.8)


plt.plot(h_values-1, excitation_energies[:, 2-1], 
             label=f'Excited State {2}',
             linestyle=line_styles[2 % len(line_styles)],  # Cycle through line styles
             marker=markers[2 % len(markers)],  # Cycle through markers
             linewidth=1.5,
             markersize=4,
             alpha=0.8)

        
            
    
plt.xlabel(r'$h$')
plt.ylabel(r'$\varepsilon_{\text{excited}}$-$\varepsilon_{\text{gs}}$')
plt.title('First two Excitation gaps vs. Magnetic Field Strength')
plt.savefig(os.path.join(directory, 'excited_states1,2.png'), dpi=400)
plt.show()


In [ ]:
print(h_values[0:26])

In [ ]:
h_values = np.linspace(0, 2, 50)

# Define line styles and markers
line_styles = ['-', '--', '-.', ':']
markers = ['o', '^', 's', '*', 'p', 'D', 'x', '+']


plt.loglog(np.abs(h_values-1)[0:26], excitation_energies[:, 2-1][0:26], 
             label=f'Excited State {2}',
             linestyle=line_styles[1 % len(line_styles)],  # Cycle through line styles
             marker=markers[1 % len(markers)],  # Cycle through markers
             linewidth=1.5,
             markersize=4,
             alpha=0.8)
a=np.log(excitation_energies[:, 2-1][0]/excitation_energies[:, 2-1][10])
b=np.log(np.abs(h_values-1)[0]/np.abs(h_values-1)[10])



print('slope = ', a/b)



plt.loglog(np.abs(h_values-1)[26:], excitation_energies[:, 1-1][26:], 
             label=f'Excited State {1}',
             linestyle=line_styles[1 % len(line_styles)],  # Cycle through line styles
             marker=markers[1 % len(markers)],  # Cycle through markers
             linewidth=1.5,
             markersize=4,
             alpha=0.8)

a=np.log(excitation_energies[:, 1-1][49]/excitation_energies[:, 1-1][49-10])
b=np.log(np.abs(h_values-1)[49]/np.abs(h_values-1)[49-10])



print('slope = ', a/b)

            
    
plt.xlabel(r'$|h-h_c|$')
plt.ylabel(r'Excitation gap $\Delta$')
plt.title('Excitation Energies vs. Magnetic Field Strength')
plt.legend()
plt.savefig(os.path.join(directory, 'loglogexcited_states1,2.png'), dpi=400)
plt.show()


## Fidelity Curve

In [ ]:
L = 20
J = 1.0
h_values = np.linspace(0, 2, 100)
periodic = True
num_states = 1
energies = np.zeros((len(h_values), num_states))


def diagonalize_h(h, num_states_to_track=2):
    H = sparseH(L, J, h, periodic)
    # Compute more than one state to track the ground state
    eigs, vecs = scipy.sparse.linalg.eigsh(H, k=num_states_to_track, which='SA')
    return eigs, vecs

# The eigensolver kept getting confused between the degenerate ground_states, So I just made it track the state during the iterations
initial_h = h_values[0]
initial_eigs, initial_vecs = diagonalize_h(initial_h, num_states_to_track=2)
vec_prev = initial_vecs[:, 0]  # The initial ground state vector

tasks = [delayed(diagonalize_h)(h) for h in h_values[1:]]

results = compute(*tasks, scheduler='processes')

fidelities = np.zeros(len(h_values) - 1)

# Calculate fidelities using state tracking
for i, (eigs, vecs) in enumerate(results):
    overlaps = np.abs(np.dot(vecs.T.conj(), vec_prev))  
    max_overlap_index = np.argmax(overlaps) 
    vec_current = vecs[:, max_overlap_index] 
    fidelity = overlaps[max_overlap_index] 
    fidelities[i] = fidelity
    vec_prev = vec_current 

I ended up still having to truncate the value of h because the solver was still getting confused for small h, need to figure out why. But the phase transition is clear though.

In [ ]:
plt.rcParams['figure.dpi'] = 400
plt.plot(h_values[:-1][13:], fidelities[13:], '-o', color='xkcd:bright red')
plt.xlabel('$h$')
plt.ylabel('Fidelity $|\\langle \\psi_{gs}(h)|\\psi_{gs}(h + \\delta h)\\rangle|$')
plt.title('Fidelity as a Function of $h$')
# plt.savefig(os.path.join(directory, 'fidelity.png'), dpi=400)
plt.show()

## 4.5 Study of magnetic ordering


In [ ]:
def correlation(vec, r):
    dim = len(vec)
    Cvec = np.zeros((dim, 1))
    
    for alpha in range(dim):
        if (alpha & (1 << (r - 1))) == (2**(r-1))*(alpha & (1 << 0)):
            Cvec[alpha] = vec[alpha]
        else:
            Cvec[alpha] = -vec[alpha]
    
    return np.dot(np.conj(vec).T, Cvec)



ferromagnetic phase

In [ ]:
H = sparseH(20, 1, 1, True)
_, vec = scipy.sparse.linalg.eigsh(H, k=1, which='SA')
print(vec.shape)

In [ ]:
R = list(range(1, 21))
Cvecs = []

for r in R:
    Cvecs.append(correlation(vec, r)[0][0])  # Extracting the correlation value from the array

print(Cvecs)

plt.plot(R, Cvecs)
plt.xlabel('r')
plt.ylabel('Correlation')
plt.title('Correlation vs r')
plt.show()
